In [131]:
from Bio.Seq import Seq
sourceseq1 = Seq("AGT");print(sourceseq1)
sourceseq2 = Seq("TC");print(sourceseq2)

AGT
TC


In [132]:
import numpy
from Bio.Seq import Seq
def transition_matrix(seq1,seq2):
    '''
    input
        seq1:
        seq2:
    output
        transition_matrix    
        
    '''
    X=len(seq1)
    Y=len(seq2)
    if X!=3:
        print("length of X is not equal to 3")
    else:
        pass
    if Y!=2:
        print("length of Y is not equal to 2")
    else:
        pass
    total_length=X+Y
    matrix_dim=3*total_length+2
    full_matrix=numpy.zeros((matrix_dim,matrix_dim))
    ####arrange row 1 (begin state)
    full_matrix[0,1:(total_length+1)]=pi_M/total_length
    full_matrix[0,(total_length+1):(2*total_length+1)]=pi_I/total_length
    ####arrange row 2:6 (from match state)
    full_matrix[1:(total_length+1),1:(total_length+1)]=rho*pi_M/total_length
    full_matrix[1:(total_length+1),(total_length+1):(2*total_length+1)]=rho*pi_I/total_length
    full_matrix[1:(total_length+1),(matrix_dim-1)]=tau
    full_matrix[1,12]=full_matrix[2,13]=delta
    full_matrix[1,2]=full_matrix[2,3]=full_matrix[4,5]=rho*pi_M/total_length+1-2*delta-rho-tau
    full_matrix[1,7]=full_matrix[2,8]=full_matrix[4,10]=rho*pi_I/total_length+delta
    full_matrix[3,1:6]=full_matrix[5,1:6]=rho*pi_M/(total_length*(rho+tau))
    full_matrix[3,6:11]=full_matrix[5,6:11]=rho*pi_I/(total_length*(rho+tau))
    full_matrix[3,(matrix_dim-1)]=full_matrix[5,(matrix_dim-1)]=tau/(rho+tau)
    ####arrange row 7:11 (from insert state)
    full_matrix[(total_length+1):(2*total_length+1),1:(total_length+1)]=rho*pi_M/total_length
    full_matrix[(total_length+1):(2*total_length+1),(total_length+1):(2*total_length+1)]=rho*pi_I/total_length
    full_matrix[(total_length+1):(2*total_length+1),(matrix_dim-1)]=tau
    full_matrix[6,1]=full_matrix[7,2]=full_matrix[9,4]=rho*pi_M/total_length+1-rho-tau-epsilon
    full_matrix[8,3]=full_matrix[10,5]=rho*pi_M/total_length+1-rho-tau
    full_matrix[6,7]=full_matrix[7,8]=full_matrix[9,10]=rho*pi_I/total_length+epsilon
    ####arrange row 12:16 (from delete state)
    full_matrix[11,2]=full_matrix[12,3]=full_matrix[14,5]=1-epsilon
    full_matrix[11,12]=full_matrix[12,13]=full_matrix[14,15]=epsilon
    return full_matrix



In [133]:
import numpy
import random, bisect
def state_transition(state_input):
    '''
    input
        one specific state
    output
        one specific state after transition based on different probs    
        
    '''
    def probchoice(items, probs, bincount=100):
        '''
        Puts items in bins in proportion to probs
        then uses random.choice() to select items.

        Larger bincount for more memory use but
        higher accuracy (on avarage). 
        '''
        bins = []
        for item,prob in zip(items, probs):
            bins += [item]*int(bincount*prob)
        return(random.choice(bins))
    loc=numpy.transpose(numpy.nonzero(T[state_input,]))
    return(probchoice(items=loc,probs=T[state_input,loc]))

In [134]:
print("Two source sequences are:")
sourceseq1 = Seq("AGT");print(sourceseq1)
sourceseq2 = Seq("TC");print(sourceseq2)
rho=0.3;delta=0.002;epsilon=0.001;pi_M=0.7;pi_I=0.3;tau=0.001
T=transition_matrix(sourceseq1,sourceseq2)
print("Related Transition matrix is:",T)


import pandas as pd #####import transition matrix
names = [_ for _ in ['B','M11','M12','M13','M21','M22','I11','I12','I13','I21','I22','D11','D12','D13','D21','D22','T']]
df = pd.DataFrame(T, index=names, columns=names)
df.to_csv('df.csv', index=True, header=True)

Two source sequences are:
AGT
TC
('Related Transition matrix is:', array([[ 0.        ,  0.14      ,  0.14      ,  0.14      ,  0.14      ,
         0.14      ,  0.06      ,  0.06      ,  0.06      ,  0.06      ,
         0.06      ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.042     ,  0.737     ,  0.042     ,  0.042     ,
         0.042     ,  0.018     ,  0.02      ,  0.018     ,  0.018     ,
         0.018     ,  0.        ,  0.002     ,  0.        ,  0.        ,
         0.        ,  0.001     ],
       [ 0.        ,  0.042     ,  0.042     ,  0.737     ,  0.042     ,
         0.042     ,  0.018     ,  0.018     ,  0.02      ,  0.018     ,
         0.018     ,  0.        ,  0.        ,  0.002     ,  0.        ,
         0.        ,  0.001     ],
       [ 0.        ,  0.13953488,  0.13953488,  0.13953488,  0.13953488,
         0.13953488,  0.05980066,  0.05980066,  0.05980066,  0.05980066,
         0.05980066,  0. 

In [136]:
def generate_targetseq():
    '''
    input
        ...
    output
        aligned sequences,emission prob is all 1/20    
        
    '''
    name_state= numpy.array([[1,"M11"],[2,"M12"],[3,"M13"],[4,"M21"],[5,"M22"],[6,"I12"],\
                         [7,"I12"],[8,"I13"],[9,"I21"],[10,"I22"],[11,"D11"],[12,"D12"],\
                             [13,"D13"],[14,"D21"],[15,"D22"]])
    Dic={1:'A',2:'G',3:'T',4:'T',5:'C'}####coming from two source seqs
    path=[]
    index=0
    while len(path) <10 and index!=16:###maximum seqence length is less than ten
        state=int(state_transition(index).tolist()[0])
        path.append(state)
        index=state
    print("path:",path)
    hidden_state=[]
    target_sequence=[]
    source_sequence=[]
    for i in path:
        hidden_state.append(name_state[i-1,1])
        if 0<i<=5:
            source_sequence.append(Dic[i])
            target_sequence.append(random.choice(list("GAVLIPFYWSTCMNQDEKRH")))
        elif 5<i<=10:
            source_sequence.append('-')
            target_sequence.append(random.choice(list("GAVLIPFYWSTCMNQDEKRH")))
        elif 10<i<=15:
            source_sequence.append(Dic[i-10])
            target_sequence.append('-')
        else:
            pass
    print("true hidden state:",hidden_state)
    print("source sequence:",source_sequence)
    print("target sequence:",target_sequence)
generate_targetseq() 

('path:', [1, 2, 3, 10, 4, 5, 9, 4, 5, 10])
('true hidden state:', ['M11', 'M12', 'M13', 'I22', 'M21', 'M22', 'I21', 'M21', 'M22', 'I22'])
('source sequence:', ['A', 'G', 'T', '-', 'T', 'C', '-', 'T', 'C', '-'])
('target sequence:', ['P', 'H', 'T', 'S', 'T', 'M', 'G', 'R', 'W', 'R'])
